In [1]:
from xgboost import XGBClassifier
import pandas as pd
import sklearn.ensemble
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score, precision_score, matthews_corrcoef, confusion_matrix, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import statistics
import numpy as np

In [2]:
df = pd.read_csv("data/4.2-hg38-paired-introns.tsv", sep="\t")
df

,chr,start,end,strand,class,GTExv2,TCGAv2,SRAv3h,RC3-Splice_site,RC3-Score,...,repeat_features_end_site:RNA repeats,repeat_features_end_site:Unknown,repeat_features_end_site:Type I Transposons/LINE,repeat_features_end_site:Type II Transposons,repeat_features_end_site:Low complexity regions,repeat_features_end_site:LTRs,repeat_features_end_site:Type I Transposons/SINE,sequence,hg38 - Acceptor splice site,hg38 - Donor splice site
0,chr1,12227,12612,+,1,1122:3199,583:1460,9374:54492,GT:AG,59151,...,0,0,0,0,0,0,0,GTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGCCGG...,GT,AG
1,chr1,12721,13220,+,1,1791:3198,783:1104,14048:56719,GT:AG,61021,...,0,0,0,0,0,0,0,GTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTAGGG...,GT,AG
2,chr1,12057,12178,+,1,NaN,NaN,NaN,NaN,-1,...,0,0,0,0,0,0,0,CACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG...,CA,GA
3,chr1,12697,12974,+,1,1:1,1:1,15:19,GT:AG,21,...,0,0,0,0,0,0,0,GTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAGTGA...,GT,AG
4,chr1,13052,13220,+,1,22:24,17:18,433:484,GC:AG,526,...,0,0,0,0,0,0,0,GCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGGAGA...,GC,AG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449562,chrY,25986748,25987810,+,0,297:602,93:135,708:1741,GT:AG,2478,...,0,0,0,0,0,0,0,GTAACAGAGTGTTTTACAGAGTAGACATTTCAACTTTCAATAAAGT...,GT,AG
449563,chrY,26335719,26337372,-,0,NaN,NaN,NaN,NaN,-1,...,0,0,0,0,0,0,0,GTGAGCCTGAACCTCTAAAAGAACAACGAGATTGGTGGCTGGGTCT...,GT,AG
449564,chrY,26337521,26354287,-,0,2:3,0:0,64:89,GT:AG,92,...,0,0,0,0,0,0,0,GTGAGTGAGTCTCTGTGTGTGTAGCTTTGCAGTATTTGTAGGAATG...,GT,AG
449565,chrY,56954656,56960285,+,0,NaN,NaN,NaN,NaN,-1,...,0,0,0,0,0,0,0,GTAAGCCATAGAAATTAGCTCTTTAAAAACCCAGAATTCTTTCTAT...,GT,AG


In [3]:
df = df.drop(columns=["GTExv2", "TCGAv2", "SRAv3h"])
df

,chr,start,end,strand,class,RC3-Splice_site,RC3-Score,repeat_features_start_site:Satellite repeats,repeat_features_start_site:Dust,repeat_features_start_site:Tandem repeats,...,repeat_features_end_site:RNA repeats,repeat_features_end_site:Unknown,repeat_features_end_site:Type I Transposons/LINE,repeat_features_end_site:Type II Transposons,repeat_features_end_site:Low complexity regions,repeat_features_end_site:LTRs,repeat_features_end_site:Type I Transposons/SINE,sequence,hg38 - Acceptor splice site,hg38 - Donor splice site
0,chr1,12227,12612,+,1,GT:AG,59151,0,0,0,...,0,0,0,0,0,0,0,GTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGCCGG...,GT,AG
1,chr1,12721,13220,+,1,GT:AG,61021,0,0,0,...,0,0,0,0,0,0,0,GTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTAGGG...,GT,AG
2,chr1,12057,12178,+,1,NaN,-1,0,0,0,...,0,0,0,0,0,0,0,CACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG...,CA,GA
3,chr1,12697,12974,+,1,GT:AG,21,0,0,0,...,0,0,0,0,0,0,0,GTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAGTGA...,GT,AG
4,chr1,13052,13220,+,1,GC:AG,526,0,0,0,...,0,0,0,0,0,0,0,GCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGGAGA...,GC,AG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449562,chrY,25986748,25987810,+,0,GT:AG,2478,0,0,0,...,0,0,0,0,0,0,0,GTAACAGAGTGTTTTACAGAGTAGACATTTCAACTTTCAATAAAGT...,GT,AG
449563,chrY,26335719,26337372,-,0,NaN,-1,0,0,0,...,0,0,0,0,0,0,0,GTGAGCCTGAACCTCTAAAAGAACAACGAGATTGGTGGCTGGGTCT...,GT,AG
449564,chrY,26337521,26354287,-,0,GT:AG,92,0,0,0,...,0,0,0,0,0,0,0,GTGAGTGAGTCTCTGTGTGTGTAGCTTTGCAGTATTTGTAGGAATG...,GT,AG
449565,chrY,56954656,56960285,+,0,NaN,-1,0,0,0,...,0,0,0,0,0,0,0,GTAAGCCATAGAAATTAGCTCTTTAAAAACCCAGAATTCTTTCTAT...,GT,AG


In [4]:
le = preprocessing.LabelEncoder()
le.fit(df["RC3-Splice_site"])
df["RC3-Splice_site"] = le.transform(df["RC3-Splice_site"])

df

,chr,start,end,strand,class,RC3-Splice_site,RC3-Score,repeat_features_start_site:Satellite repeats,repeat_features_start_site:Dust,repeat_features_start_site:Tandem repeats,...,repeat_features_end_site:RNA repeats,repeat_features_end_site:Unknown,repeat_features_end_site:Type I Transposons/LINE,repeat_features_end_site:Type II Transposons,repeat_features_end_site:Low complexity regions,repeat_features_end_site:LTRs,repeat_features_end_site:Type I Transposons/SINE,sequence,hg38 - Acceptor splice site,hg38 - Donor splice site
0,chr1,12227,12612,+,1,4,59151,0,0,0,...,0,0,0,0,0,0,0,GTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGCCGG...,GT,AG
1,chr1,12721,13220,+,1,4,61021,0,0,0,...,0,0,0,0,0,0,0,GTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTAGGG...,GT,AG
2,chr1,12057,12178,+,1,6,-1,0,0,0,...,0,0,0,0,0,0,0,CACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG...,CA,GA
3,chr1,12697,12974,+,1,4,21,0,0,0,...,0,0,0,0,0,0,0,GTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAGTGA...,GT,AG
4,chr1,13052,13220,+,1,3,526,0,0,0,...,0,0,0,0,0,0,0,GCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGGAGA...,GC,AG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449562,chrY,25986748,25987810,+,0,4,2478,0,0,0,...,0,0,0,0,0,0,0,GTAACAGAGTGTTTTACAGAGTAGACATTTCAACTTTCAATAAAGT...,GT,AG
449563,chrY,26335719,26337372,-,0,6,-1,0,0,0,...,0,0,0,0,0,0,0,GTGAGCCTGAACCTCTAAAAGAACAACGAGATTGGTGGCTGGGTCT...,GT,AG
449564,chrY,26337521,26354287,-,0,4,92,0,0,0,...,0,0,0,0,0,0,0,GTGAGTGAGTCTCTGTGTGTGTAGCTTTGCAGTATTTGTAGGAATG...,GT,AG
449565,chrY,56954656,56960285,+,0,6,-1,0,0,0,...,0,0,0,0,0,0,0,GTAAGCCATAGAAATTAGCTCTTTAAAAACCCAGAATTCTTTCTAT...,GT,AG


In [5]:
le = preprocessing.LabelEncoder()
le.fit(df["hg38 - Acceptor splice site"])
df["hg38 - Acceptor splice site"] = le.transform(df["hg38 - Acceptor splice site"])

le = preprocessing.LabelEncoder()
le.fit(df["hg38 - Donor splice site"])
df["hg38 - Donor splice site"] = le.transform(df["hg38 - Donor splice site"])

df

,chr,start,end,strand,class,RC3-Splice_site,RC3-Score,repeat_features_start_site:Satellite repeats,repeat_features_start_site:Dust,repeat_features_start_site:Tandem repeats,...,repeat_features_end_site:RNA repeats,repeat_features_end_site:Unknown,repeat_features_end_site:Type I Transposons/LINE,repeat_features_end_site:Type II Transposons,repeat_features_end_site:Low complexity regions,repeat_features_end_site:LTRs,repeat_features_end_site:Type I Transposons/SINE,sequence,hg38 - Acceptor splice site,hg38 - Donor splice site
0,chr1,12227,12612,+,1,4,59151,0,0,0,...,0,0,0,0,0,0,0,GTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGCCGG...,13,2
1,chr1,12721,13220,+,1,4,61021,0,0,0,...,0,0,0,0,0,0,0,GTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTAGGG...,13,2
2,chr1,12057,12178,+,1,6,-1,0,0,0,...,0,0,0,0,0,0,0,CACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG...,5,10
3,chr1,12697,12974,+,1,4,21,0,0,0,...,0,0,0,0,0,0,0,GTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAGTGA...,13,2
4,chr1,13052,13220,+,1,3,526,0,0,0,...,0,0,0,0,0,0,0,GCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGGAGA...,11,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449562,chrY,25986748,25987810,+,0,4,2478,0,0,0,...,0,0,0,0,0,0,0,GTAACAGAGTGTTTTACAGAGTAGACATTTCAACTTTCAATAAAGT...,13,2
449563,chrY,26335719,26337372,-,0,6,-1,0,0,0,...,0,0,0,0,0,0,0,GTGAGCCTGAACCTCTAAAAGAACAACGAGATTGGTGGCTGGGTCT...,13,2
449564,chrY,26337521,26354287,-,0,4,92,0,0,0,...,0,0,0,0,0,0,0,GTGAGTGAGTCTCTGTGTGTGTAGCTTTGCAGTATTTGTAGGAATG...,13,2
449565,chrY,56954656,56960285,+,0,6,-1,0,0,0,...,0,0,0,0,0,0,0,GTAAGCCATAGAAATTAGCTCTTTAAAAACCCAGAATTCTTTCTAT...,13,2


In [6]:
df_annot_test_y = df.pop("class")

# Drop the columns that might not be available prior to manual gene annotation or irrelevant to ML
df_annot_test_X_1 = df.drop(["chr", "start", "end", "strand", "RC3-Splice_site", "sequence"], axis=1)
del df
df_annot_test_y

0         1
1         1
2         1
3         1
4         1
         ..
449562    0
449563    0
449564    0
449565    0
449566    0
Name: class, Length: 449567, dtype: int64

In [7]:
df_annot_test_X_1

,RC3-Score,repeat_features_start_site:Satellite repeats,repeat_features_start_site:Dust,repeat_features_start_site:Tandem repeats,repeat_features_start_site:Simple repeats,repeat_features_start_site:Centromere,repeat_features_start_site:RNA repeats,repeat_features_start_site:Unknown,repeat_features_start_site:Type I Transposons/LINE,repeat_features_start_site:Type II Transposons,...,repeat_features_end_site:Centromere,repeat_features_end_site:RNA repeats,repeat_features_end_site:Unknown,repeat_features_end_site:Type I Transposons/LINE,repeat_features_end_site:Type II Transposons,repeat_features_end_site:Low complexity regions,repeat_features_end_site:LTRs,repeat_features_end_site:Type I Transposons/SINE,hg38 - Acceptor splice site,hg38 - Donor splice site
0,59151,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,13,2
1,61021,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,13,2
2,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,10
3,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,13,2
4,526,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449562,2478,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,13,2
449563,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,13,2
449564,92,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,13,2
449565,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,13,2


In [8]:
X_train_val = df_annot_test_X_1.copy()
y_train_val = df_annot_test_y.copy()
del df_annot_test_X_1
del df_annot_test_y

In [9]:
cv_acc_list = []
cv_ba_acc_list = []
cv_rocauc_list = []
cv_precision_list = []
cv_recall_list = []
cv_mcc_list = []
cv_specificity_list = []
cv_sensitivity_list = []
cv_fpr_list = []
model_dict = {}
model_index = 0
matrix = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
skf.get_n_splits(X_train_val, y_train_val)

for train_index, val_index in skf.split(X_train_val, y_train_val):
	X_train_resampled = X_train_val.iloc[train_index]
	y_train_resampled = y_train_val.iloc[train_index].to_numpy().flatten()

	X_train, X_val = X_train_resampled, X_train_val.iloc[val_index]
	y_train, y_val = y_train_resampled, y_train_val.iloc[val_index].to_numpy().flatten()

	model_dict[model_index] = XGBClassifier(seed=0)
	model_dict[model_index].fit(X_train, y_train)

	y_predict = model_dict[model_index].predict_proba(X_val)
	y_predict = y_predict[:, 1]
	y_predict_class = list(map(round, y_predict))

	test_acc = accuracy_score(y_val, y_predict_class)
	test_rocauc = roc_auc_score(y_val, y_predict)
	test_bal_acc = balanced_accuracy_score(y_val, y_predict_class)
	test_precision = precision_score(y_val, y_predict_class)  # tp/(tp+fp)
	test_recall = recall_score(y_val, y_predict_class)
	test_mcc = matthews_corrcoef(y_val, y_predict_class)
	matrix.append(confusion_matrix(y_val, y_predict_class))
	tn, fp, fn, tp = confusion_matrix(y_val, y_predict_class).ravel()

	# Adding the metrics to their list
	cv_acc_list.append(test_acc)
	cv_ba_acc_list.append(test_bal_acc)
	cv_rocauc_list.append(test_rocauc)
	cv_precision_list.append(test_precision)
	cv_mcc_list.append(test_mcc)
	cv_recall_list.append(test_recall)
	cv_specificity_list.append(tn / (tn + fp))
	cv_sensitivity_list.append(tp / (fn + tp))
	cv_fpr_list.append(fp/(fp+tn))

In [10]:
print("------------------------------------------------------------------------------------------")
print("Stratified Cross-Validation Performance (* metrics of interest)")
print("------------------------------------------------------------------------------------------")
print("*Accuracy: %s \nAUCROC: %s \n*MCC: %s \nSensitivity: %s \nSpecificity: %s \n*Precision: %s \nRecall: %s \n*False Positive Rate: %s \nBalanced Accuracy: %s" % (
        statistics.mean(cv_acc_list), statistics.mean(cv_rocauc_list), statistics.mean(cv_mcc_list),
        statistics.mean(cv_sensitivity_list), statistics.mean(cv_specificity_list), statistics.mean(cv_precision_list), statistics.mean(cv_recall_list), statistics.mean(cv_fpr_list), statistics.mean(cv_ba_acc_list)))

print("------------------------------------------------------------------------------------------")
print("Accuracy SD: %s \nAUCROC SD: %s \nMCC SD: %s \nSensitivity SD: %s \nSpecificity SD: %s \nPrecision SD: %s  \nBalanced Accuracy SD: %s" % (
        np.std(cv_acc_list), np.std(cv_rocauc_list), np.std(cv_mcc_list),
        np.std(cv_sensitivity_list), np.std(cv_specificity_list), np.std(cv_precision_list), np.std(cv_ba_acc_list)))


print("------------------------------------------------------------------------------------------")

# Print the matrix
(sum(matrix))

------------------------------------------------------------------------------------------
Stratified Cross-Validation Performance (* metrics of interest)
------------------------------------------------------------------------------------------
*Accuracy: 0.9547297727498216 
AUCROC: 0.9621058709514984 
*MCC: 0.7446834440003756 
Sensitivity: 0.983451524060864 
Specificity: 0.7093795980528186 
*Precision: 0.9665633139618882 
Recall: 0.983451524060864 
*False Positive Rate: 0.2906204019471815 
Balanced Accuracy: 0.8464155610568412
------------------------------------------------------------------------------------------
Accuracy SD: 0.00037962050749724947 
AUCROC SD: 0.00034195355011268743 
MCC SD: 0.0021929240370828582 
Sensitivity SD: 0.0005257143502265268 
Specificity SD: 0.004093571629378171 
Precision SD: 0.0004444216276813377  
Balanced Accuracy SD: 0.0018878392241286762
------------------------------------------------------------------------------------------


array([[ 33421,  13692],
       [  6660, 395794]])

# Previous model:

In [12]:
# Next we will evaluate the feature importance of the XGBoost model.
def evaluate_feature_importance(df_evaluate_feature_importance, md):
    feature_importance_average = [np.mean(i) for i in zip(*[md.get(model).feature_importances_ for model in md])]
    df_feature_importance = pd.DataFrame()
    df_feature_importance["Property"] = df_evaluate_feature_importance.columns
    df_feature_importance["Average_importance"] = feature_importance_average
    print(df_feature_importance.sort_values(by="Average_importance", ascending=False).head(50))

evaluate_feature_importance(df_evaluate_feature_importance=X_train_val.copy(), md=model_dict)

                                             Property  Average_importance
0                                           RC3-Score            0.287835
23                      repeat_features_end_site:LTRs            0.125045
11                    repeat_features_start_site:LTRs            0.087516
24   repeat_features_end_site:Type I Transposons/SINE            0.057260
25                        hg38 - Acceptor splice site            0.056587
8   repeat_features_start_site:Type I Transposons/...            0.056447
9      repeat_features_start_site:Type II Transposons            0.054162
12  repeat_features_start_site:Type I Transposons/...            0.048824
20   repeat_features_end_site:Type I Transposons/LINE            0.048421
21       repeat_features_end_site:Type II Transposons            0.048155
26                           hg38 - Donor splice site            0.029990
1        repeat_features_start_site:Satellite repeats            0.016056
13         repeat_features_end_site:Sa

# Improvements

We can see that Centromere, RNA repeats region, Unknown, and Dust region has the lowest importance, which is consistent with our hypothesis, we should be removing them from the features.